In [20]:
import pandas as pd
import numpy as np
import json
from collections import OrderedDict
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [2]:
estaciones = pd.read_csv('../data/locations/estaciones.csv')

In [3]:
estaciones = estaciones.drop(columns=['nombre', 'provincia_id'])

In [4]:
estaciones.head()

,idema,latitud,longitud,altitud
0,0002I,40.9500,0.8667,32
1,0009X,41.2000,0.9500,406
2,0016A,41.1333,1.1667,71
3,0016B,41.1500,1.1000,118
4,0034X,41.2833,1.2500,233


In [5]:
knn_features = ['latitud', 'longitud', 'altitud']

In [6]:
scaler = StandardScaler()

In [7]:
estaciones[knn_features] = scaler.fit_transform(estaciones[knn_features].values)

In [32]:
estaciones.head()

,idema,latitud,longitud,altitud
0,0002I,0.419506,1.039176,-1.090909
1,0009X,0.477403,1.055583,-0.286539
2,0016A,0.461956,1.098263,-1.007030
3,0016B,0.465823,1.085126,-0.905947
4,0034X,0.496694,1.114670,-0.658614


In [9]:
nearest_neighbors = NearestNeighbors(n_neighbors=len(estaciones), metric='euclidean')

In [10]:
nearest_neighbors.fit(estaciones[knn_features].values)

NearestNeighbors(metric='euclidean', n_neighbors=947)

In [11]:
distances, indices = nearest_neighbors.kneighbors(estaciones[knn_features].values)

In [35]:
lookup = {}

for i, row in estaciones.iterrows():
    nearest_indices = indices[i]
    closest_idemas = estaciones.iloc[nearest_indices]['idema'].tolist()
    lookup[row['idema']] = closest_idemas

In [46]:
for i in range(3):
    display(estaciones[estaciones['idema'] == lookup['0016A'][i]])

,idema,latitud,longitud,altitud
2,0016A,0.461956,1.098263,-1.00703


,idema,latitud,longitud,altitud
5,0042Y,0.458111,1.11138,-1.041442


,idema,latitud,longitud,altitud
3,0016B,0.465823,1.085126,-0.905947


In [54]:
lookup_df = pd.DataFrame.from_dict(lookup)
lookup_df = lookup_df.drop(0).reset_index(drop=True) # Ignore itself

In [55]:
lookup_df.head()

,0002I,0009X,0016A,0016B,0034X,0042Y,0061X,0066X,0073X,0076,...,C689E,C839I,C839X,C916Q,C917E,C919K,C925F,C928I,C929I,C939T
0,9981A,9775X,0042Y,0016A,9729X,0016A,9726E,0229I,0201D,0201X,...,C629Q,C839X,C839I,C316I,C928I,C929I,C916Q,C917E,C939T,C929I
1,9987P,9961X,0016B,9975X,0066X,0002I,0194D,0222X,0244X,0201D,...,C629X,C029O,C029O,C925F,C117Z,C939T,C415A,C117A,C919K,C919K
2,0042Y,9720X,0002I,0042Y,9771,0073X,9772X,0034X,0201X,0073X,...,C649I,C019V,C019V,C415A,C317B,C129Z,C455M,C117Z,C129V,C129V
3,0016A,9947X,0073X,0066X,9771C,0016B,0341,0016B,0076,0244X,...,C649R,C249I,C249I,C126A,C117A,C129V,C316I,C418L,C329Z,C329Z
4,9975X,9650X,9975X,8523X,0149X,9981A,0341X,9771,0042Y,0255B,...,C619I,C239N,C239N,C466O,C417J,C319W,C426R,C317B,C139E,C139E


In [58]:
lookup_df.to_csv('../data/ml/estacion_distance_lookup.csv', index=False)